<a href="https://colab.research.google.com/github/machlovi/BigData_Project/blob/main/Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##QUESTION 1

In [20]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
# pip install pyspark

In [23]:
import numpy as np
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import KMeans, KMeansModel
from __future__ import print_function
from pyspark import SparkContext, SparkConf
import sys
from operator import add
from pyspark.sql import SparkSession
import math

In [34]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.sql import SparkSession
conf = SparkConf().setAppName('K_Means')
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
df=spark.read.csv("/content/drive/Shareddrives/Big_Data/Lab2/shot_logs.csv",header=True,inferSchema=True).select(['player_name','SHOT_CLOCK','CLOSE_DEF_DIST','SHOT_DIST','SHOT_RESULT'])

In [35]:
df.show(4)

+-------------+----------+--------------+---------+-----------+
|  player_name|SHOT_CLOCK|CLOSE_DEF_DIST|SHOT_DIST|SHOT_RESULT|
+-------------+----------+--------------+---------+-----------+
|brian roberts|      10.8|           1.3|      7.7|       made|
|brian roberts|       3.4|           6.1|     28.2|     missed|
|brian roberts|      null|           0.9|     10.1|     missed|
|brian roberts|      10.3|           3.4|     17.2|     missed|
+-------------+----------+--------------+---------+-----------+
only showing top 4 rows



In [41]:
import pandas as pd
col=pd.read_csv('/content/drive/Shareddrives/Big_Data/Lab2/shot_logs.csv')
s = pd.Series(col.columns)
print(s)

0                        GAME_ID
1                        MATCHUP
2                       LOCATION
3                              W
4                   FINAL_MARGIN
5                    SHOT_NUMBER
6                         PERIOD
7                     GAME_CLOCK
8                     SHOT_CLOCK
9                       DRIBBLES
10                    TOUCH_TIME
11                     SHOT_DIST
12                      PTS_TYPE
13                   SHOT_RESULT
14              CLOSEST_DEFENDER
15    CLOSEST_DEFENDER_PLAYER_ID
16                CLOSE_DEF_DIST
17                           FGM
18                           PTS
19                   player_name
20                     player_id
dtype: object


In [45]:
df1=df.where(df.player_name=='brian roberts').dropna().replace(['made', 'missed'],['1','0'],'SHOT_RESULT')
# df.dropna()
# df.replace(['made', 'missed'],['1','0'],'SHOT_RESULT').show()

In [46]:
df1.show()

+-------------+----------+--------------+---------+-----------+
|  player_name|SHOT_CLOCK|CLOSE_DEF_DIST|SHOT_DIST|SHOT_RESULT|
+-------------+----------+--------------+---------+-----------+
|brian roberts|      10.8|           1.3|      7.7|          1|
|brian roberts|       3.4|           6.1|     28.2|          0|
|brian roberts|      10.3|           3.4|     17.2|          0|
|brian roberts|      10.9|           1.1|      3.7|          0|
|brian roberts|       9.1|           2.6|     18.4|          0|
|brian roberts|      14.5|           6.1|     20.7|          0|
|brian roberts|       3.4|           2.1|      3.5|          1|
|brian roberts|      12.4|           7.3|     24.6|          0|
|brian roberts|      17.4|          19.8|     22.4|          0|
|brian roberts|      16.0|           4.7|     24.5|          0|
|brian roberts|      12.1|           1.8|     14.6|          1|
|brian roberts|       4.3|           5.4|      5.9|          1|
|brian roberts|       4.4|           4.4

In [47]:
from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=['SHOT_CLOCK','CLOSE_DEF_DIST','SHOT_DIST'],
                          outputCol='features')
data=assembler.transform(df1)
data.show(4)

+-------------+----------+--------------+---------+-----------+---------------+
|  player_name|SHOT_CLOCK|CLOSE_DEF_DIST|SHOT_DIST|SHOT_RESULT|       features|
+-------------+----------+--------------+---------+-----------+---------------+
|brian roberts|      10.8|           1.3|      7.7|          1| [10.8,1.3,7.7]|
|brian roberts|       3.4|           6.1|     28.2|          0| [3.4,6.1,28.2]|
|brian roberts|      10.3|           3.4|     17.2|          0|[10.3,3.4,17.2]|
|brian roberts|      10.9|           1.1|      3.7|          0| [10.9,1.1,3.7]|
+-------------+----------+--------------+---------+-----------+---------------+
only showing top 4 rows



In [51]:
from pyspark.ml import evaluation
eval = ClusteringEvaluator(predictionCol="prediction",
                           featuresCol="features",
                           metricName="silhouette",
                           distanceMeasure="squaredEuclidean")



In [56]:
K=7
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=K, seed=1)  # 2 clusters here
model = kmeans.fit(data.select('features'))
transformed = model.transform(data)
dfGroup=transformed


centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Cluster Centers: 
[13.43076923  2.41346154  8.34423077]
[15.77763158  5.86973684 24.43421053]
[5.36923077 2.93076923 9.90384615]
[14.05641026  3.94358974 18.48333333]
[ 6.89032258  5.30107527 23.30752688]
[20.60769231  3.96923077  3.20769231]
[20.65789474 11.26315789 23.12631579]


In [57]:
dfGroup.show(4)

+-------------+----------+--------------+---------+-----------+---------------+----------+
|  player_name|SHOT_CLOCK|CLOSE_DEF_DIST|SHOT_DIST|SHOT_RESULT|       features|prediction|
+-------------+----------+--------------+---------+-----------+---------------+----------+
|brian roberts|      10.8|           1.3|      7.7|          1| [10.8,1.3,7.7]|         0|
|brian roberts|       3.4|           6.1|     28.2|          0| [3.4,6.1,28.2]|         4|
|brian roberts|      10.3|           3.4|     17.2|          0|[10.3,3.4,17.2]|         3|
|brian roberts|      10.9|           1.1|      3.7|          0| [10.9,1.1,3.7]|         0|
+-------------+----------+--------------+---------+-----------+---------------+----------+
only showing top 4 rows



In [58]:
dfGroup=cluster.orderBy("prediction", ascending=True)

In [62]:
import numpy as np
temp=np.zeros([K])

for i in range(0,K):
  x=dfGroup.where(dfGroup.prediction==i).count()
  y=dfGroup.where(dfGroup.SHOT_RESULT==1).count()
  temp[i]=float(x/y)

temp
np.argmax(temp,axis=0)

array([0.35616438, 0.52054795, 0.17808219, 0.53424658, 0.6369863 ,
       0.0890411 , 0.13013699])